In [17]:
import pandas as pd
import requests

- Import data.

In [2]:
change_frame = pd.read_csv('data/raw_changemyview_initial_scrape')
opinion_frame = pd.read_csv('data/raw_unpopularopinion_initial_scrape')

C:\Users\Avanyali\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (1,2,5,7,9,10,11,12,13,17,19,20,21,22,23,25,26,27,28,29,30,31,32,36,38,43,44,49,50,54,57,60,61,66,69,71,74,75,80,81,83,85,86,89,92,93,95,96,99,100) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Avanyali\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (1,2,5,7,9,10,11,12,17,19,20,21,22,23,25,26,27,28,29,30,31,32,36,38,42,46,47,52,56,59,61,62,63,64,66,67,68,70,72,73,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Combine frames for cleaning.

In [5]:
df = pd.concat([change_frame, opinion_frame])

- Filter out fully null columns and posts that are likely made with moderation in mind.

In [16]:
all_nas = df.isna().sum() == df.shape[0]
df.drop(columns = [key for key, value in all_nas.to_dict().items() if value], inplace = True)

In [40]:
df = df[df['pinned'] != True]
df = df[df['stickied'] != True]

- Reduce dataframe to a selection of columns.

In [47]:
relevant_columns = ['author', 'author_fullname', 'id', 'is_self', 'num_comments', 'selftext', 'score', 'subreddit', 'title']

In [48]:
df = df[relevant_columns]

- Remove posts that are not self-posted text posts.

In [51]:
df = df[df['is_self'] == True]

In [53]:
df.drop(columns = 'is_self', inplace = True)

C:\Users\Avanyali\anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


- Remove posts without body text. Confirm it does not imbalance our classes.

In [60]:
df = df[df['selftext'].isna().map(lambda x: False if x == True else True)]

In [63]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.503841
changemyview        0.496159
Name: subreddit, dtype: float64

- Check nulls.

In [64]:
df.isna().sum()

author                0
author_fullname    6443
id                    0
num_comments          0
selftext              0
score                 0
subreddit             0
title                 0
dtype: int64

In [69]:
df[df['author'] == '[deleted]']['author_fullname'].isna().sum()

3445

In [70]:
df[df['author'] == '[deleted]']['author_fullname'].shape

(3445,)

- Author fullname was hoped to be usable when author was '[deleted]'. It is not, and will be dropped.

In [71]:
df.drop(columns = 'author_fullname', inplace = True)

C:\Users\Avanyali\anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


- Drop all posts with '[deleted]' as text data, check if class imbalance is created.

In [72]:
df = df[df['selftext'] != '[deleted]']

In [74]:
df['subreddit'].value_counts(normalize = True)

unpopularopinion    0.500147
changemyview        0.499853
Name: subreddit, dtype: float64